In [1]:
import time
import requests
import pandas as pd
from tqdm import tqdm

In [4]:
data_DIR = "../../data/"
shiny_DIR = data_DIR + "shiny/"
shiny_export_DIR1 = "C:/Users/pansr/Documents/shinyNBA/data/"

In [11]:

teams_response = requests.get("https://api.pbpstats.com/get-teams/nba")
teams = teams_response.json()
teams_dict = teams["teams"]
df_teams = pd.DataFrame(teams_dict)
df_teams = df_teams.rename(columns={"text":"team"})
teams_list = df_teams["id"].to_list()
dfa = []
for year in range(2024,2025):
    season = str(year) + '-' + str(year+1)[-2:]
    for team in tqdm(teams_list):
        url = "https://api.pbpstats.com/get-team-players-for-season?S"
        params = {
            "Season": season, # To get for multiple seasons, separate seasons by comma
            "SeasonType": "Regular Season",
            "TeamId": team,
        }
        response = requests.get(url, params=params)
        response_json = response.json()
        players = response_json["players"]
        df_players1 = pd.DataFrame.from_dict(players,orient="index",columns=["player"]).reset_index()
        df_players1 = df_players1.rename(columns={"index":"id"})
        df_players1["team"] = team
        df_players1["season"] = season
        time.sleep(3)
        dfa.append(df_players1)
df_players = pd.concat(dfa)
df_players = pd.merge(df_players,df_teams,left_on="team", right_on="id")
df_players = df_players.rename(columns={"id_x":"pid","id_y":"tid","team_y":"team"})
df_players = df_players.drop(columns=["team_x"]) 
# df_players.to_parquet(shiny_DIR + "lineup_data.parquet")
# df_players.to_parquet(shiny_export_DIR1 + "lineup_data.parquet")

100%|██████████| 30/30 [01:41<00:00,  3.37s/it]
